In [1]:
from google.colab import files
uploaded = files.upload()   # Choose data_labeled.zip from your computer


Saving data_labeled.zip to data_labeled.zip


In [2]:
!unzip data_labeled.zip -d /content/


Archive:  data_labeled.zip
   creating: /content/data_labeled - Copy/
   creating: /content/data_labeled - Copy/Img_test/
  inflating: /content/data_labeled - Copy/Img_test/heart_0.png  
  inflating: /content/data_labeled - Copy/Img_test/heart_10.png  
  inflating: /content/data_labeled - Copy/Img_test/heart_1000.png  
  inflating: /content/data_labeled - Copy/Img_test/heart_1001.png  
  inflating: /content/data_labeled - Copy/Img_test/heart_1002.png  
  inflating: /content/data_labeled - Copy/Img_test/heart_1006.png  
  inflating: /content/data_labeled - Copy/Img_test/heart_1012.png  
  inflating: /content/data_labeled - Copy/Img_test/heart_1013.png  
  inflating: /content/data_labeled - Copy/Img_test/heart_1015.png  
  inflating: /content/data_labeled - Copy/Img_test/heart_1020.png  
  inflating: /content/data_labeled - Copy/Img_test/heart_1033.png  
  inflating: /content/data_labeled - Copy/Img_test/heart_1034.png  
  inflating: /content/data_labeled - Copy/Img_test/heart_1038.png  

In [3]:
!ls -R /content


/content:
'data_labeled - Copy'   data_labeled.zip   sample_data

'/content/data_labeled - Copy':
Img_test  Img_train  X_test.csv  X_train.csv  y_test.csv  y_train.csv

'/content/data_labeled - Copy/Img_test':
heart_0.png	heart_1255.png	heart_151.png  heart_439.png  heart_728.png
heart_1000.png	heart_1259.png	heart_153.png  heart_43.png   heart_731.png
heart_1001.png	heart_1261.png	heart_155.png  heart_440.png  heart_738.png
heart_1002.png	heart_1268.png	heart_158.png  heart_443.png  heart_741.png
heart_1006.png	heart_126.png	heart_161.png  heart_449.png  heart_742.png
heart_1012.png	heart_1271.png	heart_164.png  heart_450.png  heart_744.png
heart_1013.png	heart_1277.png	heart_166.png  heart_451.png  heart_745.png
heart_1015.png	heart_127.png	heart_167.png  heart_458.png  heart_746.png
heart_1020.png	heart_1283.png	heart_171.png  heart_45.png   heart_747.png
heart_1033.png	heart_1284.png	heart_173.png  heart_464.png  heart_753.png
heart_1034.png	heart_1285.png	heart_174.png  heart_46.p

In [4]:
import os
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

from PIL import Image
from sklearn.metrics import mean_squared_error

# ✅ Correct base directory
DIR = "/content/data_labeled - Copy"

X_TRAIN_PATH = f"{DIR}/X_train.csv"
X_TEST_PATH  = f"{DIR}/X_test.csv"
Y_TRAIN_PATH = f"{DIR}/y_train.csv"
Y_TEST_PATH  = f"{DIR}/y_test.csv"

IMG_TRAIN_DIR = f"{DIR}/Img_train"
IMG_TEST_DIR  = f"{DIR}/Img_test"

print("Train image directory:", IMG_TRAIN_DIR)
print("Test image directory :", IMG_TEST_DIR)

# Load CSVs
X_train_with_img = pd.read_csv(X_TRAIN_PATH)
X_test_with_img  = pd.read_csv(X_TEST_PATH)

y_train = pd.read_csv(Y_TRAIN_PATH).values.ravel()
y_test  = pd.read_csv(Y_TEST_PATH).values.ravel()

images_train = X_train_with_img["img_filename"].values
images_test  = X_test_with_img["img_filename"].values

print("Train rows:", len(images_train), "Test rows:", len(images_test))
print("First train filename:", images_train[0])


Train image directory: /content/data_labeled - Copy/Img_train
Test image directory : /content/data_labeled - Copy/Img_test
Train rows: 1000 Test rows: 500
First train filename: heart_1466.png


In [5]:
# ============================
# PART 3 – CUSTOM DATASET FOR IMAGES (COLAB)
# ============================

class CustomDataset(Dataset):
    def __init__(self, image_filenames, img_dir, targets=None):
        self.image_filenames = list(image_filenames)
        self.img_dir = img_dir
        # targets can be a numpy array, list, or pandas Series
        self.targets = None if targets is None else np.asarray(targets, dtype=np.float32)

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        # 1) Get filename and path
        fname = self.image_filenames[idx]
        img_path = os.path.join(self.img_dir, fname)

        # 2) Open image as grayscale
        img = Image.open(img_path).convert("L")  # "L" = grayscale

        # 3) Resize to 48x48 (as in P5)
        img = img.resize((48, 48))

        # 4) Convert to numpy array, normalize to [0, 1]
        img_arr = np.array(img, dtype=np.float32) / 255.0  # (48, 48)

        # 5) To tensor with channel dimension: (1, 48, 48)
        img_tensor = torch.from_numpy(img_arr).unsqueeze(0)

        # 6) Return with target if provided
        if self.targets is not None:
            y = self.targets[idx]
            return img_tensor, y
        else:
            return img_tensor


In [6]:
train_dataset = CustomDataset(
    image_filenames=images_train,
    img_dir=IMG_TRAIN_DIR,
    targets=y_train
)

print("Len train_dataset:", len(train_dataset))

img0, y0 = train_dataset[0]
print("Image tensor shape:", img0.shape)   # expect: torch.Size([1, 48, 48])
print("First target value:", y0)


Len train_dataset: 1000
Image tensor shape: torch.Size([1, 48, 48])
First target value: 0.07


In [7]:
# ============================
# PART 3 – SIMPLE CNN (P5-STYLE)
# ============================

class SimpleCNN(nn.Module):
    def __init__(self, n_features=8):
        super(SimpleCNN, self).__init__()

        # 1st conv block
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)  # 48 → 24

        # 2nd conv block
        self.conv2 = nn.Conv2d(8, 16, kernel_size=3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)  # 24 → 12

        # 3rd conv block
        self.conv3 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool2d(2, 2)  # 12 → 6

        # Fully connected layers
        self.flatten_dim = 32 * 6 * 6

        self.fc1 = nn.Linear(self.flatten_dim, 64)   # hidden representation
        self.fc2 = nn.Linear(64, n_features)         # feature vector for fusion
        self.fc_out = nn.Linear(n_features, 1)       # output risk prediction

        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.pool1(self.relu(self.conv1(x)))
        x = self.pool2(self.relu(self.conv2(x)))
        x = self.pool3(self.relu(self.conv3(x)))

        x = x.view(-1, self.flatten_dim)

        h = self.relu(self.fc1(x))
        features = self.relu(self.fc2(h))  # THIS will be extracted later

        out = self.fc_out(features)
        return out, features


In [8]:
# ============================
# PART 3 – CNN WRAPPER
# ============================

class MyCNN:
    def __init__(self, n_features=8, lr=1e-3, batch_size=32, n_epochs=15, device=None):
        self.n_features = n_features
        self.lr = lr
        self.batch_size = batch_size
        self.n_epochs = n_epochs

        self.device = device or ("cuda" if torch.cuda.is_available() else "cpu")

        self.model = SimpleCNN(n_features=n_features).to(self.device)

    # ---------------------------
    # TRAIN CNN
    # ---------------------------
    def fit(self, image_filenames, targets, img_dir):
        dataset = CustomDataset(image_filenames, img_dir, targets)
        loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=True)

        criterion = nn.MSELoss()
        optimizer = optim.Adam(self.model.parameters(), lr=self.lr)

        self.model.train()
        for epoch in range(self.n_epochs):
            total_loss = 0.0

            for imgs, y in loader:
                imgs = imgs.to(self.device)
                y = y.to(self.device).float().unsqueeze(1)

                optimizer.zero_grad()
                preds, _ = self.model(imgs)
                loss = criterion(preds, y)
                loss.backward()
                optimizer.step()

                total_loss += loss.item() * imgs.size(0)

            epoch_loss = total_loss / len(dataset)
            print(f"Epoch {epoch+1}/{self.n_epochs} - Loss: {epoch_loss:.4f}")

    # ---------------------------
    # PREDICT (used for CNN-only baseline)
    # ---------------------------
    def predict(self, image_filenames, img_dir):
        dataset = CustomDataset(image_filenames, img_dir)
        loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=False)

        self.model.eval()
        preds = []

        with torch.no_grad():
            for imgs in loader:
                imgs = imgs.to(self.device)
                out, _ = self.model(imgs)
                preds.append(out.cpu().numpy())

        preds = np.concatenate(preds).reshape(-1)
        return preds

    # ---------------------------
    # FEATURE EXTRACTION (for multimodal fusion)
    # ---------------------------
    def extract_features(self, image_filenames, img_dir):
        dataset = CustomDataset(image_filenames, img_dir)
        loader = DataLoader(dataset, batch_size=self.batch_size, shuffle=False)

        self.model.eval()
        features_all = []

        with torch.no_grad():
            for imgs in loader:
                imgs = imgs.to(self.device)
                _, feat = self.model(imgs)
                features_all.append(feat.cpu().numpy())

        features = np.concatenate(features_all, axis=0)
        return features


In [9]:
print("len(images_train):", len(images_train))
print("len(y_train):", len(y_train))

y_train_df = pd.read_csv(Y_TRAIN_PATH)
print("y_train_df.shape:", y_train_df.shape)
print(y_train_df.head())


len(images_train): 1000
len(y_train): 999
y_train_df.shape: (999, 1)
   0.06
0  0.07
1  0.09
2  0.01
3  0.26
4  0.15


In [10]:
y_train_df = pd.read_csv(Y_TRAIN_PATH)
y_test_df  = pd.read_csv(Y_TEST_PATH)

# Take the first column as the target (whatever its name is: "risk", etc.)
y_train = y_train_df.iloc[:, 0].values
y_test  = y_test_df.iloc[:, 0].values

print("len(images_train):", len(images_train))
print("len(y_train):", len(y_train))
print("len(images_test):", len(images_test))
print("len(y_test):", len(y_test))


len(images_train): 1000
len(y_train): 999
len(images_test): 500
len(y_test): 499


In [11]:
# Load targets with NO header
y_train = pd.read_csv(Y_TRAIN_PATH, header=None).iloc[:, 0].values
y_test  = pd.read_csv(Y_TEST_PATH, header=None).iloc[:, 0].values

print("len(images_train):", len(images_train))
print("len(y_train):     ", len(y_train))
print("len(images_test): ", len(images_test))
print("len(y_test):      ", len(y_test))


len(images_train): 1000
len(y_train):      1000
len(images_test):  500
len(y_test):       500


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# ---------------------------
# 1) Train/validation split
# ---------------------------
# images_train and y_train are the full training data (1000 each)
img_train_sub, img_val, y_train_sub, y_val = train_test_split(
    images_train,
    y_train,
    test_size=0.2,       # 20% for validation  (800 train / 200 val)
    random_state=42,
)

print("Train subset size:", len(img_train_sub))
print("Validation size  :", len(img_val))

# ---------------------------
# 2) Instantiate CNN
# ---------------------------
cnn = MyCNN(
    n_features=8,
    lr=1e-3,
    batch_size=32,
    n_epochs=20
)

# ---------------------------
# 3) Train ONLY on train subset
# ---------------------------
cnn.fit(
    image_filenames=img_train_sub,
    targets=y_train_sub,
    img_dir=IMG_TRAIN_DIR
)

# ---------------------------
# 4) Evaluate on validation set
# ---------------------------
y_pred_val = cnn.predict(
    image_filenames=img_val,
    img_dir=IMG_TRAIN_DIR   # still in Img_train folder
)

mse_val = mean_squared_error(y_val, y_pred_val)
rmse_val = np.sqrt(mse_val)
print("Image-only CNN – Validation RMSE:", rmse_val)
# ---------------------------
# 5) Final test evaluation
#    (still using the same trained cnn)
# ---------------------------
y_pred_test_img = cnn.predict(
    image_filenames=images_test,
    img_dir=IMG_TEST_DIR
)


mse_test = mean_squared_error(y_test, y_pred_test_img)
rmse_img_test = np.sqrt(mse_test)
print("Image-only CNN – Test RMSE:", rmse_img_test)



Train subset size: 800
Validation size  : 200
Epoch 1/20 - Loss: 0.0113
Epoch 2/20 - Loss: 0.0070
Epoch 3/20 - Loss: 0.0070
Epoch 4/20 - Loss: 0.0070
Epoch 5/20 - Loss: 0.0070
Epoch 6/20 - Loss: 0.0071
Epoch 7/20 - Loss: 0.0071
Epoch 8/20 - Loss: 0.0068
Epoch 9/20 - Loss: 0.0067
Epoch 10/20 - Loss: 0.0066
Epoch 11/20 - Loss: 0.0063
Epoch 12/20 - Loss: 0.0062
Epoch 13/20 - Loss: 0.0061
Epoch 14/20 - Loss: 0.0057
Epoch 15/20 - Loss: 0.0057
Epoch 16/20 - Loss: 0.0052
Epoch 17/20 - Loss: 0.0050
Epoch 18/20 - Loss: 0.0047
Epoch 19/20 - Loss: 0.0045
Epoch 20/20 - Loss: 0.0043
Image-only CNN – Validation RMSE: 0.06102030655099919
Image-only CNN – Test RMSE: 0.06380697089642688


In [13]:
# ============================
# STEP 4 – EXTRACT FEATURES FOR ALL IMAGES
# ============================

# 1) Train features (1000 x 8)
img_features_train = cnn.extract_features(
    image_filenames=images_train,
    img_dir=IMG_TRAIN_DIR
)

# 2) Test features (500 x 8)
img_features_test = cnn.extract_features(
    image_filenames=images_test,
    img_dir=IMG_TEST_DIR
)

print("Train image features shape:", img_features_train.shape)
print("Test  image features shape:", img_features_test.shape)


Train image features shape: (1000, 8)
Test  image features shape: (500, 8)


In [14]:
# Save as CSV (no header, just raw numbers)
np.savetxt("img_features_train.csv", img_features_train, delimiter=",")
np.savetxt("img_features_test.csv", img_features_test, delimiter=",")


In [15]:
# Save as CSV (no header, just raw numbers)
np.savetxt("img_features_train.csv", img_features_train, delimiter=",")
np.savetxt("img_features_test.csv", img_features_test, delimiter=",")


In [16]:
from google.colab import files

files.download("img_features_train.csv")
files.download("img_features_test.csv")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
from google.colab import files
import zipfile, os

# 1) Upload data_unlabeled.zip from your computer
uploaded = files.upload()   # choose data_unlabeled.zip

# 2) Unzip into /content (same as you did for data_labeled)
!unzip data_unlabeled.zip -d /content/

# 3) Inspect folders (optional, just to check paths)
!ls -R /content


Saving data_unlabeled.zip to data_unlabeled.zip
Archive:  data_unlabeled.zip
   creating: /content/data_unlabeled/
   creating: /content/data_unlabeled/Img/
  inflating: /content/data_unlabeled/Img/heart_0.png  
  inflating: /content/data_unlabeled/Img/heart_1.png  
  inflating: /content/data_unlabeled/Img/heart_10.png  
  inflating: /content/data_unlabeled/Img/heart_100.png  
  inflating: /content/data_unlabeled/Img/heart_101.png  
  inflating: /content/data_unlabeled/Img/heart_102.png  
  inflating: /content/data_unlabeled/Img/heart_103.png  
  inflating: /content/data_unlabeled/Img/heart_104.png  
  inflating: /content/data_unlabeled/Img/heart_105.png  
  inflating: /content/data_unlabeled/Img/heart_106.png  
  inflating: /content/data_unlabeled/Img/heart_107.png  
  inflating: /content/data_unlabeled/Img/heart_108.png  
  inflating: /content/data_unlabeled/Img/heart_109.png  
  inflating: /content/data_unlabeled/Img/heart_11.png  
  inflating: /content/data_unlabeled/Img/heart_110.

In [18]:
import pandas as pd

UNLABELED_DIR = "/content/data_unlabeled"
X_UNLAB_PATH  = os.path.join(UNLABELED_DIR, "X.csv")   # change if name is different
IMG_UNLAB_DIR = os.path.join(UNLABELED_DIR, "Img")

# Load tabular X for unlabeled data
X_unlab = pd.read_csv(X_UNLAB_PATH)
print(X_unlab.shape)
print(X_unlab.head())

# Column that contains the filenames (same as in labeled data)
# Change "img_filename" if your column name is different
images_unlab = X_unlab["img_filename"].values
print("Number of unlabeled images:", len(images_unlab))


(500, 14)
   age  blood pressure  calcium  cholesterol  hemoglobin  height  potassium  \
0   77           98.76     2.25        60.19       15.28    8.05       3.94   
1  129          113.66     2.47        95.99       11.34    7.36       4.88   
2  158          104.58     2.65       122.10       15.91    6.88       4.16   
3  132          122.23     2.54       112.98       14.36    6.74       4.51   
4  117           97.00     2.60       101.35       16.47    8.09       4.22   

        profession sarsaparilla smurfberry liquor smurfin donuts  vitamin D  \
0  food production     Very low               Low       Very low      27.27   
1  food production          Low         Very high           High      27.74   
2  food production     Moderate              High       Very low      31.74   
3         services          Low              High            Low      40.22   
4  food production          Low          Moderate       Moderate      30.38   

   weight img_filename  
0   82.98  hear

In [23]:
# Make sure you already ran the cells that define:
# - CustomDataset
# - SimpleCNN
# - MyCNN
# And that you already trained: my_cnn.fit(...)

# Now extract features for unlabeled images
unlab_features = cnn.extract_features(
    image_filenames=images_unlab,
    img_dir=IMG_UNLAB_DIR
)

print("Unlabeled image feature shape:", unlab_features.shape)
# Expected: (number_of_unlabeled_samples, 8)


Unlabeled image feature shape: (500, 8)


In [25]:
import numpy as np

# Build a DataFrame: keep filename + 8 feature columns
feat_cols = [f"img_feat_{i+1}" for i in range(unlab_features.shape[1])]
df_img_unlab = pd.DataFrame(unlab_features, columns=feat_cols)

# Keep filenames so you can merge later with X_unlab
df_img_unlab.insert(0, "img_filename", images_unlab)



In [26]:
# Save to CSV
out_path = "image_features_unlabeled.csv"
df_img_unlab.to_csv(out_path, index=False)
print("Saved:", out_path)

# Download to your computer (optional, for backup)
from google.colab import files
files.download(out_path)


Saved: image_features_unlabeled.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>